Ejemplo: mini análisis de notas (con def) + ranking (con lambda) - Resumen conceptual (rápido)

`def`: lógica “grande” y reusable, con validación y pasos.

`lambda`: lógica “chica” y puntual para `map`, `filter`, `sorted`.

In [3]:
estudiantes = [
    {"nombre": "Ana",   "notas": [6.0, 5.5, 6.2]},
    {"nombre": "Luis",  "notas": [4.8, 5.1, 4.9]},
    {"nombre": "Sofía", "notas": [6.8, 6.4, 6.6]},
    {"nombre": "Tomás", "notas": [3.9, 4.2, 4.0]},
]


1) `def`: calcular promedio y estado (aprobado/reprobado)

Esta función:

- valida que existan notas

- calcula promedio

- determina estado según umbral

- devuelve un diccionario con resultados

In [5]:
def resumen_estudiante(estudiante, umbral_aprobacion=4.0):
    notas = estudiante["notas"]

    # Validación simple
    if len(notas) == 0:
        return {
            "nombre": estudiante["nombre"],
            "promedio": None,
            "estado": "SIN NOTAS"
        }

    promedio = sum(notas) / len(notas)

    if promedio >= umbral_aprobacion:
        estado = "APROBADO"
    else:
        estado = "REPROBADO"

    return {
        "nombre": estudiante["nombre"],
        "promedio": round(promedio, 2),
        "estado": estado
    }
print(resumen_estudiante(estudiantes[0]))  # Ana
print(resumen_estudiante(estudiantes[3]))  # Tomás

{'nombre': 'Ana', 'promedio': 5.9, 'estado': 'APROBADO'}
{'nombre': 'Tomás', 'promedio': 4.03, 'estado': 'APROBADO'}


2) `lambda`: aplicar la función a todos + ordenar por promedio

In [11]:
# (a) Transformar: crear una lista de resúmenes

resumenes = list(map(lambda e: resumen_estudiante(e), estudiantes))
print(resumenes)

# (b) Ordenar: ranking por promedio (descendente)

ranking = sorted(
    resumenes,
    key=lambda r: (r["promedio"] is None, r["promedio"]),  # None al final
    reverse=True
)

for r in ranking:
    print(r["nombre"], r["promedio"], r["estado"])

# 3) lambda extra: filtro de aprobados

aprobados = list(filter(lambda r: r["estado"] == "APROBADO", resumenes))
print(aprobados)


[{'nombre': 'Ana', 'promedio': 5.9, 'estado': 'APROBADO'}, {'nombre': 'Luis', 'promedio': 4.93, 'estado': 'APROBADO'}, {'nombre': 'Sofía', 'promedio': 6.6, 'estado': 'APROBADO'}, {'nombre': 'Tomás', 'promedio': 4.03, 'estado': 'APROBADO'}]
Sofía 6.6 APROBADO
Ana 5.9 APROBADO
Luis 4.93 APROBADO
Tomás 4.03 APROBADO


# Limpieza de Datos — Real Estate

Esta notebook está diseñada para estudiantes que recién comienzan con **Python + Pandas**.

**Meta:** dejar el dataset **consistente** y **listo para análisis**.

Trabajaremos paso a paso:
1. Cargar datos
2. Inspección rápida
3. Estandarizar valores faltantes
4. Corregir tipos (números vs texto)
5. Validar categorías simples
6. Completar (imputar) valores faltantes
7. Chequeos finales y guardado


## 1) Importar librerías

Usaremos solo lo esencial: `pandas` y `numpy`.

In [1]:
import pandas as pd
import numpy as np

A partir del contexto del negocio (corretaje de propiedades), una expectativa razonable para este dataset es:

| Columna | Significado | Tipo esperado | Observaciones |
|---|---|---|---|
| `PID` | Property ID | Entero (ID) | Idealmente único y no nulo |
| `ST_NUM` | Street number | Entero | Puede faltar en algunos registros |
| `ST_NAME` | Street name | Texto (string) | Normalizado (mayúsculas, sin espacios extra) |
| `OWN_OCCUPIED` | Propiedad ocupada por dueño | Categórica (`Y`/`N`) | Valores fuera del dominio se consideran inválidos |
| `NUM_BEDROOMS` | Número de dormitorios | Entero | Puede venir como texto / tokens no-estándar |
| `NUM_BATH` | Número de baños | Numérico (float) | Puede venir como texto inválido |
| `SQ_FT` | Metros/pies cuadrados | Numérico (entero) | Puede venir como texto / tokens no-estándar |

En las siguientes secciones vamos a contrastar esta expectativa contra los datos reales.


## 2) Cargar el archivo CSV

En este proyecto el archivo se llama `real-estate.csv`.

In [3]:
path = "real-estate.csv"
df = pd.read_csv(path)

df.head()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600


## 3) Inspección rápida

Antes de limpiar, conviene responder:
- ¿Cuántas filas/columnas hay?
- ¿Qué tipos detectó Pandas?
- ¿Hay valores raros (como `--`, `n/a`, etc.)?

In [4]:
print("Filas, Columnas:", df.shape)
print("Columnas:")
print(df.columns)


Filas, Columnas: (9, 7)
Columnas:
Index(['PID', 'ST_NUM', 'ST_NAME', 'OWN_OCCUPIED', 'NUM_BEDROOMS', 'NUM_BATH',
       'SQ_FT'],
      dtype='object')


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PID           8 non-null      float64
 1   ST_NUM        7 non-null      float64
 2   ST_NAME       9 non-null      object 
 3   OWN_OCCUPIED  8 non-null      object 
 4   NUM_BEDROOMS  7 non-null      object 
 5   NUM_BATH      8 non-null      object 
 6   SQ_FT         8 non-null      object 
dtypes: float64(2), object(5)
memory usage: 636.0+ bytes


## 4) Buscar valores faltantes (missing) *tal como vienen*

Pandas detecta algunos faltantes (por ejemplo `NaN`), pero a veces los CSV traen faltantes como texto:
- `--`
- `n/a`
- `NA`
- string vacío

Primero veamos cuántos faltantes hay por columna.

In [6]:
df.isna().sum()

PID             1
ST_NUM          2
ST_NAME         0
OWN_OCCUPIED    1
NUM_BEDROOMS    2
NUM_BATH        1
SQ_FT           1
dtype: int64

## 5) Estandarizar faltantes no estándar

Convertiremos varios “faltantes en texto” a `NaN`.

Nota: esta es una práctica muy común en limpieza de datos.

In [7]:
missing_tokens = ["--", "n/a", "na", "NA", "NaN", ""]

# Reemplazamos solo en columnas tipo texto/mixtas (igual funciona en todas)
df = df.replace(missing_tokens, np.nan)

df.head()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,NaN
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600


Volvemos a contar faltantes, ahora que estandarizamos.

In [8]:
df.isna().sum()

PID             1
ST_NUM          2
ST_NAME         0
OWN_OCCUPIED    1
NUM_BEDROOMS    3
NUM_BATH        1
SQ_FT           2
dtype: int64

## 6) Eliminar duplicados (si existen)

Primero revisamos cuántas filas duplicadas hay, y si existen las eliminamos.

In [9]:
duplicadas = df.duplicated().sum()
print("Filas duplicadas:", duplicadas)

# Si hay duplicadas, las quitamos
if duplicadas > 0:
    df = df.drop_duplicates()

print("Filas, Columnas (después de duplicados):", df.shape)


Filas duplicadas: 0
Filas, Columnas (después de duplicados): (9, 7)


## 7) Revisar una columna clave: `PID`

En este dataset, `PID` parece ser un identificador.

Si un registro no tiene `PID`, podría ser una idea **eliminarlo** (porque no se puede identificar correctamente).

In [10]:
print("Faltantes en PID:")
print(df["PID"].isna().sum())

# Eliminamos filas sin PID
df = df.dropna(subset=["PID"]).copy()

print("Filas, Columnas (después de eliminar PID faltante):", df.shape)


Faltantes en PID:
1
Filas, Columnas (después de eliminar PID faltante): (8, 7)


## 8) Convertir columnas numéricas a número

Algunas columnas deberían ser numéricas (por ejemplo cantidad de dormitorios), pero vienen con textos.

Usaremos `pd.to_numeric(..., errors='coerce')`:
- Convierte lo posible
- Lo que no se puede, lo transforma en `NaN`

In [11]:
numeric_cols = ["ST_NUM", "NUM_BEDROOMS", "NUM_BATH", "SQ_FT"]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Revisamos tipos nuevamente
print(df.dtypes)

df.head()

PID             float64
ST_NUM          float64
ST_NAME          object
OWN_OCCUPIED     object
NUM_BEDROOMS    float64
NUM_BATH        float64
SQ_FT           float64
dtype: object


,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1.0,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,NaN
2,100003000.0,NaN,LEXINGTON,N,NaN,1.0,850.0
3,100004000.0,201.0,BERKELEY,12,1.0,NaN,700.0
5,100006000.0,207.0,BERKELEY,Y,NaN,1.0,800.0


## 9) Validar la variable categórica `OWN_OCCUPIED`

Esperamos valores como `Y` o `N`. Si aparece algo raro (por ejemplo `12`), lo convertimos a `NaN`.

In [12]:
# Normalizamos a mayúsculas y quitamos espacios
df["OWN_OCCUPIED"] = df["OWN_OCCUPIED"].astype(str).str.strip().str.upper()

# Convertimos los "NAN" (string) a NaN real
# (esto puede ocurrir porque convertimos a str arriba)
df.loc[df["OWN_OCCUPIED"].isin(["NAN", "NONE"]), "OWN_OCCUPIED"] = np.nan

# Dejamos solo Y / N; lo demás -> NaN
df.loc[~df["OWN_OCCUPIED"].isin(["Y", "N"]) , "OWN_OCCUPIED"] = np.nan

print(df["OWN_OCCUPIED"].value_counts(dropna=False))

OWN_OCCUPIED
Y      4
N      2
NaN    2
Name: count, dtype: int64


## 10) Revisar faltantes *después* de conversiones

Normalmente al convertir números aparecen nuevos `NaN` (porque había textos en columnas numéricas).

In [13]:
df.isna().sum()

PID             0
ST_NUM          2
ST_NAME         0
OWN_OCCUPIED    2
NUM_BEDROOMS    3
NUM_BATH        2
SQ_FT           2
dtype: int64

## 11) Completar (imputar) valores faltantes

Para un primer curso, una estrategia simple y común:
- Variables **numéricas**: reemplazar por la **mediana**
- Variables **categóricas**: reemplazar por la **moda** (el valor más frecuente)

Esto no siempre es lo “mejor” en un proyecto real, pero es una base sólida para empezar.

In [14]:
# 11.1) Numéricas -> mediana
for col in numeric_cols:
    med = df[col].median()
    df[col] = df[col].fillna(med)
    print(f"{col}: mediana usada = {med}")

# 11.2) Categóricas -> moda
cat_cols = ["ST_NAME", "OWN_OCCUPIED"]
for col in cat_cols:
    moda = df[col].mode(dropna=True)
    if len(moda) > 0:
        moda = moda.iloc[0]
        df[col] = df[col].fillna(moda)
        print(f"{col}: moda usada = {moda}")

# Verificamos faltantes
print("Faltantes por columna (después de imputación):")
print(df.isna().sum())


ST_NUM: mediana usada = 204.0
NUM_BEDROOMS: mediana usada = 2.0
NUM_BATH: mediana usada = 1.0
SQ_FT: mediana usada = 900.0
ST_NAME: moda usada = BERKELEY
OWN_OCCUPIED: moda usada = Y
Faltantes por columna (después de imputación):
PID             0
ST_NUM          0
ST_NAME         0
OWN_OCCUPIED    0
NUM_BEDROOMS    0
NUM_BATH        0
SQ_FT           0
dtype: int64


## 12) Chequeo simple de rangos y valores

Un chequeo básico: mirar estadísticas descriptivas y detectar si algo queda "fuera de rango".

In [15]:
df.describe(include="all")

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
count,8.000000e+00,8.000000,8,8,8.000000,8.000000,8.000000
unique,NaN,NaN,5,2,NaN,NaN,NaN
top,NaN,NaN,LEXINGTON,Y,NaN,NaN,NaN
freq,NaN,NaN,2,6,NaN,NaN,NaN
mean,1.000050e+08,193.125000,NaN,NaN,2.000000,1.187500,987.500000
std,2.927700e+03,36.494373,NaN,NaN,0.755929,0.372012,340.954542
min,1.000010e+08,104.000000,NaN,NaN,1.000000,1.000000,700.000000
25%,1.000028e+08,200.000000,NaN,NaN,1.750000,1.000000,837.500000
50%,1.000050e+08,204.000000,NaN,NaN,2.000000,1.000000,900.000000
75%,1.000072e+08,208.500000,NaN,NaN,2.250000,1.125000,962.500000


## 13) (Opcional) Outliers con IQR en `SQ_FT`

Esto es un método común y relativamente sencillo:
- Calculamos Q1 (25%) y Q3 (75%)
- IQR = Q3 - Q1
- Límite inferior = Q1 - 1.5*IQR
- Límite superior = Q3 + 1.5*IQR

Aquí solo **contamos** cuántos outliers hay. No los eliminaremos automáticamente.

In [16]:
q1 = df["SQ_FT"].quantile(0.25)
q3 = df["SQ_FT"].quantile(0.75)
iqr = q3 - q1

lim_inf = q1 - 1.5 * iqr
lim_sup = q3 + 1.5 * iqr

outliers = df[(df["SQ_FT"] < lim_inf) | (df["SQ_FT"] > lim_sup)]

print("Q1:", q1)
print("Q3:", q3)
print("IQR:", iqr)
print("Límite inferior:", lim_inf)
print("Límite superior:", lim_sup)
print("Cantidad de outliers (SQ_FT):", len(outliers))

outliers

Q1: 837.5
Q3: 962.5
IQR: 125.0
Límite inferior: 650.0
Límite superior: 1150.0
Cantidad de outliers (SQ_FT): 1


,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
8,100009000.0,215.0,TREMONT,Y,2.0,2.0,1800.0


## 14) Guardar dataset limpio

Guardaremos un nuevo archivo CSV con el sufijo `_clean_basic`.

In [17]:
out_path = "real-estate_clean_basic.csv"
df.to_csv(out_path, index=False)

print("Archivo guardado en:", out_path)


Archivo guardado en: real-estate_clean_basic.csv


## 15) Resultado final

A modo de cierre, revisamos:
- primeras filas
- tipos
- si quedan faltantes

In [18]:
df.head()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3.0,1.0,1000.0
1,100002000.0,197.0,LEXINGTON,N,3.0,1.5,900.0
2,100003000.0,204.0,LEXINGTON,N,2.0,1.0,850.0
3,100004000.0,201.0,BERKELEY,Y,1.0,1.0,700.0
5,100006000.0,207.0,BERKELEY,Y,2.0,1.0,800.0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0 to 8
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PID           8 non-null      float64
 1   ST_NUM        8 non-null      float64
 2   ST_NAME       8 non-null      object 
 3   OWN_OCCUPIED  8 non-null      object 
 4   NUM_BEDROOMS  8 non-null      float64
 5   NUM_BATH      8 non-null      float64
 6   SQ_FT         8 non-null      float64
dtypes: float64(5), object(2)
memory usage: 512.0+ bytes


In [20]:
df.isna().sum()

PID             0
ST_NUM          0
ST_NAME         0
OWN_OCCUPIED    0
NUM_BEDROOMS    0
NUM_BATH        0
SQ_FT           0
dtype: int64

Valores Atípicos
> pptx

---
# Obtención de Datos

- Archivos de texto plano
- Archivos Excel
- Tablas Web
- Archios HDF5
- Bases de Datos
- APIs REST

In [2]:
import pandas as pd

### Archivos de texto plano

In [22]:
# leer un archivo csv
df = pd.read_csv('notas-alumnos.csv')
df

,colegio,sexo,edad,ausencias,N1,N2,N3
0,GP,F,18,6,1.8,2.1,2.1
1,GP,F,17,4,1.8,1.8,2.1
2,GP,F,15,10,2.4,2.8,3.5
3,GP,F,15,2,5.2,4.9,5.2
4,GP,F,16,4,2.1,3.5,3.5
...,...,...,...,...,...,...,...
390,MS,M,20,11,3.2,3.2,3.2
391,MS,M,17,3,4.9,5.6,5.6
392,MS,M,21,3,3.5,2.8,2.4
393,MS,M,18,0,3.9,4.2,3.5


### Planillas Excel

In [27]:
# leer planilla
df = pd.read_excel('sueldos.xlsx')
df.head(2)

,_id,NOMBRE,TITULO Y/O ESPECIALIDAD,LABOR,LUGAR DE SU FUNCION,SUELDO LIQUIDO
0,1,Cecilia Del Carmen Ayala Cabrera,Enfermera,Encargada Del Cecosf,Cecosf Padre Hugo Cornelissen,1.803.344
1,2,Jesús Ignacio Contreras Vivar,Tec. Enfermería,"Despacho De Medicamentos, Pnac Pacam",Farmacia/Coordinación,236.489


In [28]:
# leer una hoja especifica de una planilla
df = pd.read_excel('sueldos.xlsx', sheet_name='Hoja Principal')
df.head(2)

,_id,NOMBRE,TITULO Y/O ESPECIALIDAD,LABOR,LUGAR DE SU FUNCION,SUELDO LIQUIDO
0,1,Cecilia Del Carmen Ayala Cabrera,Enfermera,Encargada Del Cecosf,Cecosf Padre Hugo Cornelissen,1.803.344
1,2,Jesús Ignacio Contreras Vivar,Tec. Enfermería,"Despacho De Medicamentos, Pnac Pacam",Farmacia/Coordinación,236.489


In [29]:
# escribir una planilla
df[['NOMBRE','SUELDO LIQUIDO']].to_excel('resumen_sueldos.xlsx')

In [30]:
# escribir una planilla sin indice y con nombre de hoja
df[['NOMBRE','SUELDO LIQUIDO']].to_excel('resumen_sueldos.xlsx', index=False,
                                         sheet_name='Hoja Resumen')

In [31]:
df = pd.read_excel('resumen_sueldos.xlsx')
df.head()

,NOMBRE,SUELDO LIQUIDO
0,Cecilia Del Carmen Ayala Cabrera,1.803.344
1,Jesús Ignacio Contreras Vivar,236.489
2,Carolina Andrea Estay Pangue,664.647
3,Jorge Eduardo García Lagos,1.279.353
4,Carolina Lissett Gómez Morales,255.036


### Lectura de páginas html

In [3]:
url = 'https://es.wikipedia.org/wiki/Regiones_de_Chile'
tables = pd.read_html(url)

HTTPError: HTTP Error 403: Forbidden

In [5]:
import pandas as pd
import requests

url = "https://es.wikipedia.org/wiki/Regiones_de_Chile"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8",
}

html = requests.get(url, headers=headers, timeout=30).text
tables = pd.read_html(html)

print("Tablas encontradas:", len(tables))
tables[0].head()


Tablas encontradas: 5


/tmp/ipykernel_7039/746734708.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


,Región,Región.1,Región.2
0,Provincia (1928-1974),Provincia (1928-1974),Provincia (1928-1974)
1,Arica y Parinacota Tarapacá Antofagasta Atacam...,Arica y Parinacota Tarapacá Antofagasta Atacam...,Arica y Parinacota Tarapacá Antofagasta Atacam...
2,País,Chile,Chile
3,Creados por,Junta Militar de Gobierno,Junta Militar de Gobierno
4,Fecha de creación,1974,1974


In [6]:
tables[1]

,Regiones Corfo,Provincias (hasta 1976),Regiones actuales (desde 2018)
0,Norte Grande,Tarapacá y Antofagasta,"Arica y Parinacota, Tarapacá y Antofagasta"
1,Norte Chico,Atacama y Coquimbo,Atacama y Coquimbo
2,Núcleo Central,"Aconcagua, Valparaíso, Santiago, O'Higgins, Co...","Valparaíso, Metropolitana de Santiago, O'Higgi..."
3,Concepción y La Frontera,"Concepción, Arauco, Biobío, Malleco y Cautín",Biobío y La Araucanía
4,Región de los Lagos,"Valdivia, Osorno y Llanquihue",Los Ríos y Los Lagos (continental)
5,Región de los Canales,"Chiloé, Aysén y Magallanes","Los Lagos (Provincia de Chiloé), Aysén y Magal..."


### Bases de Datos

In [8]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [9]:
# creamos una conexion
conn = create_engine('sqlite:///:memory:')

In [10]:
# creamos un dataframe de ejjemplo
df = pd.DataFrame({ 'id': range(1000,1010),
                    'Ingresos':np.random.randint(1e5,9e5,10),
                    'Egresos':np.random.randint(1e5,9e5,10)})
df.head()

,id,Ingresos,Egresos
0,1000,459472,692913
1,1001,135845,118030
2,1002,248163,149541
3,1003,477521,173545
4,1004,319593,642008


In [11]:
# guardamos dataframe en una tabla sql
df.to_sql('mi_tabla', conn)

10

In [12]:
query = 'SELECT * FROM mi_tabla WHERE Ingresos < 300000'

# realizamos consulta sql en la base de datos
pd.read_sql(query, conn)

,index,id,Ingresos,Egresos
0,1,1001,135845,118030
1,2,1002,248163,149541
2,6,1006,148023,172905
3,9,1009,275936,639708


### Lectura de una API

In [ ]:
import requests

In [ ]:
url = 'https://jsonplaceholder.typicode.com/posts'

headers = {"Accept":"*/*","Content-Type":"application/json"}
response = requests.get(url, headers=headers)

if response.status_code != 200:
        raise IOError(f'Error Code: {response.status_code}. Reason: {response.reason}')

In [ ]:
# respuesta json
json_resp = response.json()

In [ ]:
# transformamos en dataframe
df = pd.json_normalize(json_resp)
df.head()